# Attribute Descriptors

A descriptor is a class that implements a dynamic protocol consisting of the __get__,
__set__, and __delete__ methods.

### Descriptor Example: Attribute Validation

### LineItem Take #3: A Simple Descriptor
As we said in the introduction, a class implementing a __get__, a __set__, or a
__delete__ method is a descriptor. You use a descriptor by declaring instances of it
as class attributes of another class.

In [2]:
class Quantity:
    def __init__(self, storage_name):
        self.storage_name = storage_name
    def __set__(self, instance, value):
        if value > 0:
            instance.__dict__[self.storage_name] = value
        else:
            msg = f'{self.storage_name} must be > 0'
            raise ValueError(msg)
    def __get__(self, instance, owner):
        return instance.__dict__[self.storage_name]

In [3]:
class House:
    rooms = Quantity('number_of_rooms')

In [4]:
def __get__(self, instance, owner):
    if instance is None:
        return self
    else:
        return instance.__dict__[self.storage_name]

In [5]:
class LineItem:
    weight = Quantity('weight')
    price = Quantity('price')
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    def subtotal(self):
        return self.weight * self.price

LineItem Take #4: Automatic Naming of Storage Attributes

In [12]:
class Quantity2:
    def __set_name__(self, owner, name):
        self.storage_name = name
    def __set__(self, instance, value):
        if value > 0:
            instance.__dict__[self.storage_name] = value
        else:
            msg = f'{self.storage_name} must be > 0'
            raise ValueError(msg)

# no __get__ needed
class LineItem2:
    weight = Quantity2()
    price = Quantity2()
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    def subtotal(self):
        return self.weight * self.price


In [18]:
obj1 = LineItem2('abi', 10, 10)

### LineItem Take #5: A New Descriptor Type

Example 23-5. model_v5.py: the Validated ABC

In [19]:
import abc
class Validated(abc.ABC):
    def __set_name__(self, owner, name):
        self.storage_name = name
    def __set__(self, instance, value):
        value = self.validate(self.storage_name, value)
        instance.__dict__[self.storage_name] = value
    @abc.abstractmethod
    def validate(self, name, value):
        """return validated value or raise ValueError"""

In [22]:
class VQuantity(Validated):
    """a number greater than zero"""
    def validate(self, name, value):
        if value <= 0:
            raise ValueError(f'{name} must be > 0')
        return value


class NonBlank(Validated):
    """a string with at least one non-space character"""
    def validate(self, name, value):
        value = value.strip()
        if not value:
            raise ValueError(f'{name} cannot be blank')
        return value

In [24]:
class LineItem3:
    description = NonBlank()
    weight = VQuantity()
    price = VQuantity()
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    def subtotal(self):
        return self.weight * self.price

In [33]:
obh = LineItem3('abi', 10, 3)

In [34]:
obh.subtotal()

30

Example 23-8. descriptorkinds.py: simple classes for studying descriptor overriding
behaviors

In [35]:
def cls_name(obj_or_cls):
    cls = type(obj_or_cls)
    if cls is type:
        cls = obj_or_cls
    return cls.__name__.split('.')[-1]

In [36]:
def display(obj):
    cls = type(obj)
    if cls is type:
        return f'<class {obj.__name__}>'
    elif cls in [type(None), int]:
        return repr(obj)
    else:
        return f'<{cls_name(obj)} object>'

In [37]:
def print_args(name, *args):
    pseudo_args = ', '.join(display(x) for x in args)
    print(f'-> {cls_name(args[0])}.__{name}__({pseudo_args})')

In [38]:
class Overriding:
    """a.k.a. data descriptor or enforced descriptor"""
    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)
    def __set__(self, instance, value):
        print_args('set', self, instance, value)

In [39]:
class OverridingNoGet:
    """an overriding descriptor without ``__get__``"""
    def __set__(self, instance, value):
        print_args('set', self, instance, value)

In [40]:
class NonOverriding:
    """a.k.a. non-data or shadowable descriptor"""
    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)

In [41]:
class Managed:
    over = Overriding()
    over_no_get = OverridingNoGet()
    non_over = NonOverriding()
    def spam(self):
        print(f'-> Managed.spam({display(self)})')

In [42]:
obj = Managed()

In [43]:
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [44]:
Managed.over

-> Overriding.__get__(<Overriding object>, None, <class Managed>)


In [45]:
obj.over = 7

-> Overriding.__set__(<Overriding object>, <Managed object>, 7)


In [46]:
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [47]:
obj.__dict__['over'] = 8

In [48]:
vars(obj)

{'over': 8}

In [49]:
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [50]:
obj.__dict__['over'] = 9

In [51]:
vars(obj)

{'over': 9}

In [52]:
obj.__dict__['abc'] = 9

In [53]:
vars(obj)

{'over': 9, 'abc': 9}

Example 23-10. Overriding descriptor without __get__

In [54]:
obj.over_no_get

In [55]:
Managed.over_no_get

In [56]:
obj.over_no_get = 7
obj.over_no_get

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)


In [60]:
obj.__dict__['over_no_get'] = 9
obj.over_no_get

9

In [59]:
obj.over_no_get = 7
obj.over_no_get

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)


9